In [1]:
import cv2
import json
import math
import numpy as np
from os.path import join
from matplotlib import pyplot as plt

Path to JSON file containing classifications, created with [RunwayML](https://runwayml.com/)

In [2]:
inputFile = '../training/Annotation Group - Images and Colour Bars for BSO Examples.json'

Directory of original images

In [3]:
fileDir = '../training/data/images/'

Directory to store training images

In [4]:
outputDir = '../training/data/labels/'

Load classifications

In [5]:
with open(inputFile, 'r') as f:
    data = json.load(f)

Define colours for categories

In [6]:
colours = [(0,0,255),
          (0,255,0),
          (255,0,0),
          (0,255,255),
          (255,255,0),
          (255,0,255)]
categoryColours = {}
for i, category in enumerate(data['categories']):
    categoryColours[category['id']] = colours[i]

Define order to draw categories

In [7]:
categoryOrder = ['Document', 'Colour Bar', 'Image']

In [8]:
categoryIdOrder = [[c['id'] for c in data['categories'] if c['name'] == d][0] for d in categoryOrder]

Draw and save training images

In [9]:
files = data['files']
for file in files.keys():
    filePath = join(fileDir, file)
    inputImage = cv2.imread(filePath)
    outputImage = np.zeros(inputImage.shape, np.uint8)
    for annotation in sorted(files[file], key=lambda x: categoryIdOrder.index(x['categoryId'])):
        if annotation['type'] == 'BOUNDING_BOX':
            # Fix coordinates of bounding box (seem to be misinterpreteds)
            topLeftX = math.floor(annotation['boundingBox'][0] * inputImage.shape[1])
            topLeftY = math.floor(annotation['boundingBox'][1] * inputImage.shape[0])
            bottomRightX = math.ceil(annotation['boundingBox'][2] * inputImage.shape[1])
            bottomRightY = math.ceil(annotation['boundingBox'][3] * inputImage.shape[0])
            outputImage = cv2.rectangle(outputImage, (topLeftX, topLeftY), (bottomRightX, bottomRightY), categoryColours[annotation['categoryId']], -1)
            outputFilePath = join(outputDir, file)
            cv2.imwrite(outputFilePath, outputImage)
